In [1]:
import torch
from allennlp.models.archival import load_archive
from allennlp.predictors.predictor import Predictor
from allennlp.common.file_utils import cached_path
import torch.nn.functional as F

import pandas as pd
import swifter

In [2]:
import sys
sys.path.insert(0,'../')

In [3]:
from src.tweet_reader import *
from src.disaster_classifier import *
from src.predictor import *

In [4]:
archive = load_archive('../tmp/model.tar.gz')
predictor = Predictor.from_archive(archive, 'disaster', dataset_reader_to_load='tweety')


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/puneet/Envs/venv/lib/python3.8/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


In [5]:
df = pd.read_csv("../data/test-v2.csv")

In [12]:
inputs={'text':'I love volcano erupting!'}
instance = predictor._json_to_instance(inputs)

In [23]:
keys = archive.model.vocab.get_token_to_index_vocabulary('tokens').keys()

In [13]:
instance.fields['tokens'].tokens

[i, love, volcano, erupting]

In [ ]:
def predict(row):
    text = row['combined_text']
    sentence = {'text': text}
    outputs = predictor.predict_json(sentence)
    return outputs['label']
    
df['target'] = df.swifter.apply(lambda x: predict(x),1)


In [ ]:
sentence = 'I  love Berlin!'
predict({'combined_text': sentence})

# Submission

In [36]:
def combine(test_file, output_file):
    test_file = pd.read_csv(test_file)
    output_file = pd.read_json(output_file,lines=True)

    submissions = pd.DataFrame({'id': test_file['id']})
    submissions['target'] = output_file.label.values
    
    submissions['target_prob']= output_file.probs.apply(lambda x: x[1],1)

    return submissions

In [68]:
df1 = combine("../data/test/test.csv","../data/output.csv")
#df2 = combine("../data/test/test.csv","../data/output2.csv")
df2 = combine("../data/test/test.csv","../data/output3.csv")

In [33]:
(df1.target != df2.target).sum()

331

In [69]:
final_df = df2

In [77]:
final_df['target_prob'] = (0.7*df1.target_prob + 0.3*df2.target_prob)

In [78]:
(final_df['target_prob'] > 0.5).sum()

1163

In [79]:
final_df['target'] = final_df['target_prob'].apply(lambda x: 1 if x>=0.5 else 0)

In [80]:
final_df.head(100)

,id,target,target_prob
0,0,1,0.848268
1,2,1,0.678455
2,3,1,0.862347
3,9,1,0.634202
4,11,1,0.842759
...,...,...,...
95,323,0,0.148551
96,324,0,0.126938
97,325,0,0.187173
98,326,0,0.331998


In [62]:
df_test = pd.read_csv("../data/test/test.csv")

In [81]:
final_df[['id','target']].to_csv('submission_elmo_accmetric.csv',index=False)